# Randomized Cholesky QR


While [CholeskyQR](./cholesky-qr.ipynb) has a high flop efficiency, it is unstable due to forming the Gram matrix $\vec{A}^\T\vec{A}$.


## Subspace Embedding for approximate QR


```{prf:algorithm} Randomized approximate QR
:label: sketched-qr

**Input:** $\vec{A}\in\R^{n\times d}$, sketching dimension $k$

1. Sample $\vec{S}\sim\Call{Sketch}(k,n)$
1. For $\vec{Y} = \vec{S}\vec{A}$
1. Compute QR factorization $\vec{Q}'\vec{R} = \Call{qr}(\vec{Y})$.
1. Form $\vec{Q} = \vec{A}\vec{R}^{-1}$

**Output:** $\vec{Q}, \vec{R}$
```


````{prf:theorem} Well-conditioned basis from subspace embedding

Suppose $\vec{A}$ is full-ranka and $\vec{S}$ is an $\varepsilon$-subspace embedding for $\vec{A}$.
Then the output $\vec{Q}$ of {prf:ref}`sketched-qr` is well-conditioned in the sense that 
```{math}
\cond(\vec{Q}) \leq \frac{1+\varepsilon}{1-\varepsilon}.
```
````


````{admonition} Proof
:class: dropdown

By definition, 
```{math}
\forall \vec{x}\in\range(\vec{A}): (1-\varepsilon)\|\vec{x}\|_2 \leq \|\vec{S}\vec{x}\|_2 \leq (1+\varepsilon)\|\vec{x}\|_2.
```
We can reparameterize $\vec{x} = \vec{A}\vec{R}^{-1}\vec{c} = \vec{Q}\vec{c}$ for some $\vec{c}\in\R^d$.
Since $\vec{S}\vec{A}\vec{R}^{-1} = \vec{Q}'$ has orthonormal columns, we have $\|\vec{S}\vec{x}\| = \|\vec{S}\vec{A}\vec{R}^{-1}\vec{c}\|_2 = \|\vec{c}\|_2$.
Thus,
```{math}
(1-\varepsilon) \|\vec{Q}\vec{c}\|_2 \leq \|\vec{c}\|_2 \leq (1+\varepsilon)\|\vec{Q}\vec{c}\|_2.
```
Hence, 
```{math}
\smin(\vec{Q}) = \min_{\vec{c}\neq 0} \frac{\|\vec{Q}\vec{c}\|_2}{\|\vec{c}\|_2} \geq \frac{1}{1+\varepsilon} 
,\qquad
\smax(\vec{Q}) = \max_{\vec{c}\neq 0} \frac{\|\vec{Q}\vec{c}\|_2}{\|\vec{c}\|_2} \leq \frac{1}{1-\varepsilon}.
````


Unfortunately, [oblivious sketching methods](../Sketching/subspace-embedding.ipynb) require $k$ to grow polynomially with $\varepsilon$. 
As such, {prf:ref}`sketched-qr` can easily provide a well-conditioned basis, it is not suitable for providing a nearly machine-precision orthonormal basis.


## Preconditioning for QR


We can use the Cholesky QR method to orthonalize the output of {prf:ref}`sketched-qr`.
Since this output can bet guaranteed to be well-conditioned, the stability issues of Cholesky QR are mitigated.


```{prf:algorithm} Randomized Cholesky QR
:label: randomized-cholesky-qr

**Input:** $\vec{A}\in\R^{n\times d}$, sketching dimension $k$

1. Compute approximate QR factorization $\vec{Q}_1,\vec{R}_1 = \Call{rand-qr}(\vec{A})$
1. $\vec{Q},\vec{R}_1 = \Call{cholesky-qr}(\vec{Q}_1)$
1. $\vec{R} = \vec{R}_2\vec{R}_1$

**Output:** $\vec{Q}, \vec{R}$
```

## Numerical Experiment

Let's compare the performance and accuracy of different QR factorization methods, including our new randomized Cholesky QR approach.

In [17]:
import numpy as np
import scipy as sp
import time
import pandas as pd

def sparse_stack(n,k,zeta):

    k_loc = (k-1)//zeta + 1
    k = k_loc * zeta


    C = np.random.randint(0,k_loc,size=(n,zeta))
    C += np.arange(0,k_loc*zeta,k_loc)

    indices = C.flatten()
    values = ((2/np.sqrt(zeta))*np.random.randint(2,size=n*zeta) -1/np.sqrt(zeta))
    indptr = np.arange(0,n+1)*zeta
    S = sp.sparse.csc_matrix ((values,indices,indptr),shape=(k,n))

    return S

In [2]:
# Generate a random matrix with controlled condition number
n = 5000
d = 300

U, s, Vt = np.linalg.svd(np.random.rand(n, d), full_matrices=False)
s = np.geomspace(1e-4, 1, d)  # Controlled singular values for numerical stability
A = U @ np.diag(s) @ Vt

def cholesky_QR(A):
    """
    QR factorization using Cholesky decomposition
    """
    R = np.linalg.cholesky(A.T @ A).T
    Q = sp.linalg.solve_triangular(R.T, A.T, lower=True).T
    return Q, R

In [24]:
def sketched_qr(A,k,zeta):

    n, d = A.shape

    S = sparse_stack(n,k,4)  # Generate sign sketch
    Y = S @ A

    R = np.linalg.qr(Y, mode='r')    # QR factorization of sketched matrix

    
    Q1 = sp.linalg.solve_triangular(R.T,A.T,lower=True).T
    
    return Q1, R

def randomized_cholesky_QR(A,k,zeta):

    Q1, R1 = sketched_qr(A,k,zeta)  # Compute sketched QR factorization for preconditioning
    Q2, R2 = cholesky_QR(Q1) # Apply Cholesky QR to well-conditioned matrix Q1
    R = R2 @ R1 # Combine the R factors
    
    return Q2, R

In [25]:
# Define QR factorization methods
qr_methods = {
    'Householder QR': {
        'func': lambda: np.linalg.qr(A, mode='reduced'),
    },
    'Cholesky QR': {
        'func': lambda: cholesky_QR(A),
    },
    'Randomized Cholesky QR': {
        'func': lambda: randomized_cholesky_QR(A,k=int(1.25*d),zeta=4),
    }
}

In [26]:
# Time the QR factorization methods
n_repeat = 10  # Number of repetitions for averaging

results = []

for method_name, method_info in qr_methods.items():
    # Time the method
    start = time.time()
    for _ in range(n_repeat):
        Q, R = method_info['func']()
    end = time.time()
    
    avg_time = (end - start) / n_repeat
    
    # Compute accuracy metrics
    results.append({
        'method': method_name,
        'time (s)': avg_time,
        'orthogonality': np.linalg.norm(Q.T @ Q - np.eye(d)),
        'reconstruction': np.linalg.norm(A - Q @ R)
    })

# Create DataFrame and compute relative performance
results_df = pd.DataFrame(results)
results_df['speedup'] = results_df['time (s)'] / results_df['time (s)'].max()

# Display results with formatting
results_df.reindex(columns=['method','time (s)','speedup','orthogonality','reconstruction']).style.format({
    'time (s)': '{:.4f}',
    'orthogonality': '{:1.1e}',
    'reconstruction': '{:1.1e}',
    'speedup': '{:.1f}x',
})

,method,time (s),speedup,orthogonality,reconstruction
0,Householder QR,0.1078,1.0x,7.0e-15,2.7e-15
1,Cholesky QR,0.0239,0.2x,4.2e-09,6.7e-16
2,Randomized Cholesky QR,0.0514,0.5x,2.1e-14,3.8e-15


The results demonstrate the trade-offs between the different QR factorization approaches:

- **Householder QR**: Slow but numerically stable with excellent orthogonality
- **Cholesky QR**: Fast but numerically unstable due to the Gram matrix $\vec{A}^\T\vec{A}$
- **Randomized Cholesky QR**: Achieves a middle ground - faster than Householder QR while maintaining much better numerical stability than standard Cholesky QR

The randomized approach successfully addresses the numerical instability of Cholesky QR by using the sketching step to precondition the matrix, making it well-conditioned before applying the Cholesky factorization.